# 가격 예측을 위한 스태킹(Stacking) 모델 v2
*메타 모델로 LinearRegression과 Ridge를 비교하여 최적의 스태킹 조합을 찾습니다.*

In [6]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import GradientBoostingRegressor, StackingRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import warnings

warnings.filterwarnings('ignore')

## 1. 데이터 불러오기 및 통합 전처리

In [ ]:
# 데이터 로드 및 전처리 (이전과 동일)
cold_df = pd.read_excel('../data/mon_cold.xlsx')
wind_df = pd.read_excel('../data/mon_wind.xlsx')
hot_df = pd.read_excel('../data/mon_hot.xlsx')
price_df = pd.read_excel('../data/region_price.xlsx')
weather_df = pd.read_csv('../data/region_weather.csv')
trade_df = pd.read_excel('../data/spinach_cucumber_df.xlsx')

cold_df_melted = cold_df.melt(id_vars=['지역'], var_name='날짜', value_name='한파발생')
wind_df_melted = wind_df.melt(id_vars=['지역'], var_name='날짜', value_name='태풍발생')
hot_df_melted = hot_df.melt(id_vars=['지역'], var_name='날짜', value_name='폭염발생')
for df in [price_df, weather_df, trade_df]:
    df['날짜'] = pd.to_datetime(df['날짜'], errors='coerce').dt.strftime('%Y-%m')
merged_df = pd.merge(price_df, weather_df, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, cold_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, wind_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, hot_df_melted, on=['지역', '날짜'], how='left')
merged_df = pd.merge(merged_df, trade_df, on=['품목', '날짜'], how='left')
merged_df.dropna(subset=['평균가격'], inplace=True)
merged_df = merged_df[merged_df['평균가격'] > 0].copy()
merged_df.fillna(0, inplace=True)

merged_df['날짜'] = pd.to_datetime(merged_df['날짜'])
merged_df['연도'] = merged_df['날짜'].dt.year
merged_df['월'] = merged_df['날짜'].dt.month
print('데이터 통합 및 전처리 완료.')

데이터 통합 및 전처리 완료.


## 2. 스태킹 모델 (메타 모델 비교) 튜닝 및 평가

In [ ]:
def train_and_evaluate_stacking(df, vegetable_name):
    target_df = df[df['품목'] == vegetable_name].copy()
    target_df['지역'] = target_df['지역'].astype('category').cat.codes

    features = ['평균기온(°C)', '월합강수량(00~24h만)(mm)', '평균풍속(m/s)', '최심적설(cm)', '한파발생', '태풍발생', '폭염발생', '지역', '연도', '월', '수출중량', '수입중량']
    target = '평균가격'

    X = target_df[features]
    y = target_df[target]
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    # 1. 기본 모델 정의
    estimators = [
        ('gb', GradientBoostingRegressor(random_state=42)),
        ('xgb', XGBRegressor(random_state=42)),
        ('lgbm', LGBMRegressor(random_state=42, verbose=-1))
    ]

    # 2. 스태킹 모델 생성 (메타 모델: LinearRegression)
    stacking_model = StackingRegressor(estimators=estimators, final_estimator=LinearRegression())

    # 3. 하이퍼파라미터 그리드 정의 (각 기본 모델의 파라미터를 지정)
    params = {
        'gb__n_estimators': [100, 200],
        'gb__learning_rate': [0.05, 0.1],
        'xgb__n_estimators': [100, 200],
        'xgb__learning_rate': [0.05, 0.1],
        'lgbm__n_estimators': [100, 200],
        'lgbm__learning_rate': [0.05, 0.1],
        'final_estimator__fit_intercept': [True, False]
    }

    # 4. GridSearchCV를 사용한 모델 튜닝 및 학습
    print(f'--- {vegetable_name} 스태킹 모델 튜닝 시작 ---')
    grid_search = GridSearchCV(estimator=stacking_model, param_grid=params, cv=3, n_jobs=-1, scoring='r2', verbose=2)
    grid_search.fit(X_train, y_train)

    # 5. 최적 모델로 평가
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)

    # 6. 최종 결과 출력
    print(f'--- {vegetable_name} 최종 스태킹 모델 평가 결과 ---')
    print(f'MAE: {mae:.4f}')
    print(f'MSE: {mse:.4f}')
    print(f'RMSE: {rmse:.4f}')
    print(f'R-squared: {r2:.4f}')
    print(f'Best Hyperparameters: {grid_search.best_params_}')
    print('-'*80)

# 함수 실행
train_and_evaluate_stacking(merged_df, '시금치')
train_and_evaluate_stacking(merged_df, '오이')

--- 시금치 스태킹 모델 튜닝 시작 ---
Fitting 3 folds for each of 128 candidates, totalling 384 fits
--- 시금치 최종 스태킹 모델 평가 결과 ---
MAE: 2759.3627
MSE: 32759543.3088
RMSE: 5723.5953
R-squared: 0.7188
Best Hyperparameters: {'final_estimator__fit_intercept': False, 'gb__learning_rate': 0.05, 'gb__n_estimators': 200, 'lgbm__learning_rate': 0.1, 'lgbm__n_estimators': 200, 'xgb__learning_rate': 0.05, 'xgb__n_estimators': 100}
--------------------------------------------------------------------------------
--- 오이 스태킹 모델 튜닝 시작 ---
Fitting 3 folds for each of 128 candidates, totalling 384 fits
--- 오이 최종 스태킹 모델 평가 결과 ---
MAE: 2566.4443
MSE: 13090299.3208
RMSE: 3618.0519
R-squared: 0.8639
Best Hyperparameters: {'final_estimator__fit_intercept': False, 'gb__learning_rate': 0.1, 'gb__n_estimators': 200, 'lgbm__learning_rate': 0.1, 'lgbm__n_estimators': 200, 'xgb__learning_rate': 0.05, 'xgb__n_estimators': 200}
--------------------------------------------------------------------------------
